# Importação das bibliotecas necessárias

In [ ]:
!pip install opencv-python
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cluster import KMeans

# Método HOG (Histograma Orientado ao Gradiente)
* Metodo onde recebe uma imagem e retorna o descritor HOG que é o histograma orientado ao gradiente.
* O HOG é um descritor de características, ele vai generalizar a imagem de uma forma que os algoritmos de aprendizado possam dizer o que é e o que não é uma imagem

In [207]:
def hog(img):
    bin_n = 16
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)

    bins = np.int32(bin_n*ang/(2*np.pi))
 #método para extrair o descritor das imagens
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)
    return hist

# Método de Redimensionamento
* Redimensionando a imagem para um tamanho fixo e, em seguida, 'alisando' a imagem em uma lista de intensidades de pixels brutas

In [208]:
def redimensionamento (image, size = (32, 32)):
    return cv2.resize (image, size).flatten ()

# Leitura dos Dados

* Inserindo as imagens do database em 'imagens'
* Inserindo os descritores das imagens redimensionadas em 'hogdata'
* Atribuindo resultados a uma matriz em 'X'

In [209]:
#glob é usado Converte o caminho p para um caminho absoluto canônico

imagens = [cv2.imread(file) for file in sorted(glob.glob("database/*.png"))]
hogdata = []
for i in range(len(imagens)):
    imagem = redimensionamento(imagens[i])
    hogdata.append(hog(imagem))
X = np.float32(hogdata).reshape(-1,64)

# Rotulando as labels
* Rotulando as imagens como verdadeiras e falsas

In [210]:
labels = np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1])

# Dividindo os dados para treino e teste
* 75% para treino
* 25% para teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=0)

# KNN
* Realizando Treino e testes com o KNN
* Informando a taxa de acerto do teste

In [211]:
knn = KNeighborsClassifier () 

knn.fit(X_train, y_train)
acc = knn.score(X_test, y_test)
print("Acurácia: {:.2f}%".format(acc*100))

Acurácia: 64.29%


# SVM
* Realizando treino e testes com o SVM
* Informando a taxa de acerto do teste

In [215]:
svc = svm.SVC(kernel='linear', C=1, gamma=1) 
svc.fit (X_train, y_train)
acc = svc.score(X_test, y_test)

print("Acurácia: {:.2f}%".format(acc*100))

Acurácia: 50.00%


# K-Means
* Realizando agrupamento com o K-Means
* Informando a taxa de acerto

In [229]:
kmeans = KMeans(n_clusters = 2, random_state=0)
kmeans.fit (X_train)

acc = accuracy_score (kmeans.predict(X_test), y_test)
print("Acurácia: {:.2f}%".format(acc*100))

Acurácia: 42.86%
